## EDA and data preparation code development

### Northwestern Banking

#### Loan prediction project

In [1]:
# Initiate Libraries

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import datetime

import os


In [2]:
# Import CSV
datapath = os.path
df = pd.read_csv("smallerdata.csv",  sep=',')
# df = pd.read_excel("smallerdata.csv")

C:\Users\jlsud\anaconda3\envs\NWU2\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (1,7,20,36,50,60,119,130,131,132,135,136,137,140,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Remove unnecessary columns

In [6]:
# Remove column created for random numbers - used to random sample larger dataset

del df['Unnamed: 0']

In [7]:
# Remove column with website address for loan

del df['url']

###  'id' column contains summary rows
To eliminate these summary rows, the variable must first be recast from object to string. 
Each summary row contains the word 'amount'. We will use this to elminiate these rows.

Finally, the 'id' variable will be converted to an integer

In [ ]:
df['id'].dtype

In [8]:
# Convert all of 'id' to string type first to allow next cell to drop rows 

df['id']=df['id'].astype(str)

In [9]:
# Remove summary rows

df = df[~df['id'].str.contains("amount")]

In [10]:
df['id'].dtype

dtype('O')

In [11]:
df['id'].describe()

count        265292
unique       265292
top       149406288
freq              1
Name: id, dtype: object

In [12]:
# convert 'id' to integer

df['id']=df['id'].astype(np.int32)

In [13]:
df['id'].describe()

count    2.652920e+05
mean     9.094560e+07
std      4.871346e+07
min      5.891500e+04
25%      5.484879e+07
50%      9.603417e+07
75%      1.350207e+08
max      1.596103e+08
Name: id, dtype: float64

In [14]:
# Reset index after eliminating the summary rows

df = df.reset_index(drop=True)

### Convert Date Variables
This data set contains numerous date variables that are all stored as object. 
The format for the dates is inconsistent. A function was developed to deal with these
variables.

In [ ]:
df['issue_d'].dtype

In [ ]:
# Updated Date Convert on 9-Oct-2020

def dateconvert(datestring):

    #empty 
    if type(datestring) == float:
        year = "1900"
        month = "Jan"
    #3-Jan for 01/01/2003
    elif len(datestring)==5:
        year = "0" + datestring[0:1]
        month = datestring[2:5]
    #19-Mar for 03/01/2019
    elif datestring[0:2].isnumeric():
        year = datestring[0:2]
        month = datestring[3:6]
    #Feb-2000 for 02/01/2000    
    elif datestring[4:8].isnumeric() and len (datestring[4:8])==4:
        month = datestring[0:3]
        year = datestring[4:8]
    #Feb-01 for 02/01/2019
    elif datestring[4:8].isnumeric() and len (datestring[4:8])==2:
        month = datestring[0:3]
        year = datestring[4:6]
    
    date_time_str = month +' 01 '+ year
    #date format switches partiall through the dataset to not include 4 char year
    if len(year)==2:
        date_time_obj = datetime.datetime.strptime(date_time_str, '%b %d %y')  
    else:
        date_time_obj = datetime.datetime.strptime(date_time_str, '%b %d %Y')
    
    return date_time_obj

In [15]:
# List of columns for data transformation

datecol_list = ['issue_d','earliest_cr_line','last_pymnt_d','last_credit_pull_d',
                'hardship_start_date','hardship_end_date','payment_plan_start_date',
                'debt_settlement_flag_date','settlement_date']

In [16]:
for c in range(0,len(datecol_list)):
    df[datecol_list[c]+"2"] = df.apply(lambda x: dateconvert(x[datecol_list[c]]), axis =1)

In [17]:
df.dtypes

id                                                     int32
member_id                                            float64
loan_amnt                                            float64
funded_amnt                                          float64
funded_amnt_inv                                      float64
term                                                  object
int_rate                                              object
installment                                          float64
grade                                                 object
sub_grade                                             object
emp_title                                             object
emp_length                                            object
home_ownership                                        object
annual_inc                                           float64
verification_status                                   object
issue_d                                               object
loan_status             

### Convert Interest Rate from String with % sign

In [ ]:
df['issue_d'].head()

In [18]:
# convert 'int_rate' to string

df['int_rate']=df['int_rate'].astype(str)

In [19]:
# strip off % sign and convert to float

df['int_rate'] = df['int_rate'].str.rstrip('%').astype('float') / 100.0

In [20]:
df['int_rate'].head()

0    0.1199
1    0.0849
2    0.1288
3    0.1344
4    0.1288
Name: int_rate, dtype: float64

### EDA helpers

In [ ]:
# This is a quick and dirty function for quick reviews of variables 
# Example cell follows

# Works on numerical. Categoricals will still return number missing and description but will then throw error. 
# NOT A GREAT function but it saved me some cut and paste time

def analyze_var(var):
    print(var.describe())
    print('\n')
    print('number missing:')
    print(sum(var.isnull()))

    sns.distplot(var, color='g', bins=100)

In [ ]:
# Example - dataframe['variable']

analyze_var(df['settlement_term'])

In [ ]:
# Found this on web Good way to review our variables post clean
# https://stackoverflow.com/questions/37366717/pandas-print-column-name-with-missing-values

def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero Missing Values'] = 100 * mz_table['Total Zero Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
#         mz_table.to_excel('D:/sampledata/missing_and_zero_values.xlsx', freeze_panes=(1,0), index = False)
        return mz_table

missing_zero_values_table(df)